In [ ]:
import cv2
import numpy as np
from IPython.display import display
from PIL import Image
import os
from os.path import join

def d(i):
    display(Image.fromarray(cv2.convertScaleAbs(i)))


In [ ]:

hr_folder = "E:/Dataset/amakano2/1440p_dat"
lr_folder = "E:/Dataset/amakano2/720p"
hr_cut_folder = "E:/Dataset/amakano2/hr_freq_cut"
lr_cut_folder = "E:/Dataset/amakano2/lr_freq_cut"
hr_cut_folder = "R:/hr"
lr_cut_folder = "R:/lr"

img_size = int(1024 / 2)
img_num = 2


def high_freq(img):
    ret, img = cv2.threshold(out, 20, 255, cv2.THRESH_BINARY)
    img = cv2.GaussianBlur(img, (29, 29), 0)
    ret ,img = cv2.threshold(out, 127, 255, cv2.THRESH_BINARY)    
    return img


for root, dirs, files in os.walk(lr_folder):
    for file in files:
        if "png" in file or "webp" in file:
            base, ext = os.path.splitext(file)
            img_hr = cv2.imread(join(hr_folder, file))
            img_lr = cv2.imread(join(lr_folder, file))
            # rgb = cv2.cvtColor(img_hr, cv2.COLOR_BGR2RGB)  # Converting BGR to RGB
            gray = cv2.cvtColor(img_lr, cv2.COLOR_BGR2GRAY)  # Converting BGR to gray
            laplacian = cv2.Laplacian(gray, cv2.CV_64F)
            out = np.abs(laplacian)
            out = high_freq(out)
            for i in range(img_num):
                convolved = cv2.filter2D(
                    out,
                    -1,
                    np.ones((img_size, img_size), dtype=np.float64),
                    anchor=(0, 0),
                    borderType=cv2.BORDER_CONSTANT,
                )
                y, x = np.unravel_index(np.argmax(convolved), convolved.shape)
                if x > img_lr.shape[1] - img_size:
                    print(f"{base}_{i}.png {x=} {img_lr.shape[1]=}")
                    x = img_lr.shape[1] - img_size
                if y > img_lr.shape[0] - img_size:
                    print(f"{base}_{i}.png {y=} {img_lr.shape[0]=}")
                    y = img_lr.shape[0] - img_size
                print(f"{base}_{i}.png {np.max(convolved)}")
                out[y : y + img_size, x : x + img_size] = 0
                cutted_hr = img_hr[y*2 : y*2 + img_size*2, x*2 : x*2 + img_size*2]
                cutted_lr = img_lr[y : y + img_size, x : x + img_size]
                cv2.imwrite(join(hr_cut_folder, f"{base}_{i}.png"), cutted_hr)
                cv2.imwrite(join(lr_cut_folder, f"{base}_{i}.png"), cutted_lr)

In [1]:
# cut to piece
import cv2
import os
from os.path import join

raw_folder = "E:/Dataset/amakano2/screenshot/hr"
cutted_folder = "E:/Dataset/amakano2/screenshot/hr_cut"
origin_size = 1440

# raw_folder = "E:/Dataset/amakano2/screenshot/lr/"
# cutted_folder = "E:/Dataset/amakano2/screenshot/lr_cut"
# origin_size = 720


img_size = int(origin_size / 18) * 9

x_slice = [i * int(origin_size / 18) for i in [0, 8, 15, 23]]
y_slice = [i * int(origin_size / 18) for i in [0, 9]]

for root, dirs, files in os.walk(raw_folder):
    for file in files:
        if "png" in file or "webp" in file:
            base, ext = os.path.splitext(file)
            img = cv2.imread(join(root, file))
            for i, x in enumerate(x_slice):
                for j, y in enumerate(y_slice):
                    cut = img[y : y + img_size, x : x + img_size]
                    cv2.imwrite(join(cutted_folder, f"{base}_{i+j*4}.png"), cut)